Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

sns.set_style("whitegrid")

Importar base de dados, definir colunas de interesse e medidas descritivas

In [ ]:
df = pd.read_csv("../databases/ENEM_2023_FINAL_num.csv")

cluster_columns = ['TP_DEPENDENCIA_ADM_ESC', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'ESCOLARIDADE_PAI', 'ESCOLARIDADE_MAE', 'INTERNET_CASA', 'EST_RENDA_PER_CAP', 'EST_CELULAR_PER_CAP', 'EST_COMP_PER_CAP']

df_cluster = df[cluster_columns]

display(df_cluster.sample(5))
display(df_cluster.describe().T)

Visualização dos outliers

In [ ]:
def gerar_boxplot(columns:list[str], title:str):
    plt.figure(figsize=(13, 6))
    plt.title(title)
    sns.boxplot(data=df_cluster[columns], orient='v', palette="Set2")
    plt.show()


notas_columns = ['NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']
gerar_boxplot(notas_columns, "Distribuição das Notas (com outliers)")

socioeconomicas1_columns = ['EST_CELULAR_PER_CAP', 'EST_COMP_PER_CAP']
gerar_boxplot(socioeconomicas1_columns, "Distribuição de Atributos Socieconômicos I (com outliers)")

socioeconomicas2_columns = ['EST_RENDA_PER_CAP']
gerar_boxplot(socioeconomicas2_columns, "Distribuição de Atributos Socieconômicos II (com outliers)")

Overview dos outliers

In [ ]:
def overview_outliers_for_column(df:pd.DataFrame, col:str) -> dict:    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)

    inf = Q1 - 1.5 * (Q3 - Q1)
    sup = Q3 + 1.5 * (Q3 - Q1)
    outliers_inf = df[df[col] < inf].shape[0]
    outliers_sup = df[df[col] > sup].shape[0]

    return {
        "COLUNA": col,
        "LIMITE_INF": inf,
        "LIMITE_SUP": sup,
        "N_OUTLIERS_INF": outliers_inf,
        "N_OUTLIERS_SUP": outliers_sup,
        "TOTAL_OUTLIERS": outliers_inf + outliers_sup
    }

def overview_outliers(df:pd.DataFrame, columns:list[str]):
    results = [overview_outliers_for_column(df, col) for col in columns]
    return pd.DataFrame(results).set_index("COLUNA")

columns = ['NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'EST_RENDA_PER_CAP', 'EST_CELULAR_PER_CAP', 'EST_COMP_PER_CAP']
outliers_overview = overview_outliers(df_cluster, columns)

display(outliers_overview)

Marcar e remover os outliers

In [ ]:
def remover_outliers_tukey(df: pd.DataFrame, colunas: list[str]) -> pd.DataFrame:
    outlier_geral_mask = pd.Series(False, index=df.index)
    
    for col in colunas:
        if col not in df.columns:
            continue
            
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        
        mask_outlier_col = (df[col] < limite_inferior) | (df[col] > limite_superior)
        
        outlier_geral_mask = outlier_geral_mask | mask_outlier_col

    df_sem_outliers = df[~outlier_geral_mask].copy()
    
    return df_sem_outliers

columns = ['NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'NU_NOTA_REDACAO', 'EST_RENDA_PER_CAP', 'EST_CELULAR_PER_CAP', 'EST_COMP_PER_CAP']
df_cluster_clean = remover_outliers_tukey(df_cluster, columns)
display(df_cluster_clean)

Padronizar escala

In [ ]:
def padronizar_zscore(df:pd.DataFrame, columns:list[str]):
    scaler = StandardScaler()

    dados_padronizados_np = scaler.fit_transform(df[columns])

    df_padronizado = pd.DataFrame(
        dados_padronizados_np,
        columns=columns,
        index=df.index
    )
    
    return df_padronizado, scaler

df_padronizado, scaler = padronizar_zscore(df_cluster_clean, columns)

Método Hierarquico

In [ ]:
def plotar_dendrograma(df_padronizado:pd.DataFrame):
    Z = linkage(df_padronizado, method='ward', metric='euclidean')

    plt.figure(figsize=(13, 6))
    plt.title('Dendrograma (Método Ward - Distância Euclidiana)')
    plt.xlabel('Índice da Observação')
    plt.ylabel('Distância Euclidiana')
    
    dendrogram(
        Z,
        leaf_rotation=90.,  # Gira os rótulos do eixo X
        leaf_font_size=8.,  # Tamanho da fonte
        truncate_mode='lastp',
        p=50,
        show_leaf_counts=True,
        color_threshold=None # A cor padrão azul é aplicada a todo o gráfico
    )
    plt.show()
    
    return Z

Z_linkage = plotar_dendrograma(df_padronizado)

Métodos não Hierarquicos

In [ ]:
def plotar_elbow_method(df_padronizado: pd.DataFrame, max_k: int = 10):
    inertia_scores = []
    
    # Testar de K=1 (para começar o gráfico) até K=max_k
    K_range = range(1, max_k + 1)
    
    # Rodar K-Means para cada K
    for k in K_range:
        # random_state garante que os resultados sejam reproduzíveis
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(df_padronizado)
        inertia_scores.append(kmeans.inertia_)
        
    # Plotar
    plt.figure(figsize=(10, 6))
    plt.plot(K_range, inertia_scores, marker='o', linestyle='--')
    plt.title('Método do Cotovelo (Elbow Method) para Otimização de K')
    plt.xlabel('Número de Clusters (K)')
    plt.ylabel('Inércia (WGSS)')
    plt.xticks(K_range)
    plt.show()
    
    return inertia_scores

inertia_scores = plotar_elbow_method(df_padronizado, max_k=10)

In [ ]:
def plotar_silhouette_score(df_padronizado: pd.DataFrame, max_k: int = 10):
    silhouette_scores = {}
    
    # Testar de K=2 (score não é definido para K=1) até K=max_k
    K_range = range(2, max_k + 1)

    # Rodar K-Means e calcular o Score para cada K
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(df_padronizado)
        score = silhouette_score(df_padronizado, labels)
        silhouette_scores[k] = score
        print(f"K={k}: Silhouette Score = {score:.4f}")
        
    # Plotar
    plt.figure(figsize=(10, 6))
    plt.bar(silhouette_scores.keys(), silhouette_scores.values())
    plt.title('Coeficiente de Silhueta (Silhouette Score) por K')
    plt.xlabel('Número de Clusters (K)')
    plt.ylabel('Silhouette Score')
    plt.xticks(list(K_range))
    plt.show()
    
    return silhouette_scores

# Exemplo de Uso
silhouette_results = plotar_silhouette_score(df_padronizado, max_k=10)

K-Means

In [ ]:
K_FINAL = 2
kmeans_final = KMeans(n_clusters=K_FINAL, random_state=42, n_init=10)
kmeans_final.fit(df_padronizado)

# 2. Atribuir o Cluster ao DataFrame original
# Usamos o índice do df_padronizado para garantir que a atribuição seja correta
df_cluster_clean['CLUSTER'] = kmeans_final.labels_

print("="*50)
print(f"ETAPA 7.3 CONCLUÍDA: Atribuição dos {K_FINAL} Clusters")
print(f"Distribuição dos Clusters:")
print(df_cluster_clean['CLUSTER'].value_counts())
print("="*50)

Análise dos perfis

In [ ]:
columns = ['NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'NU_NOTA_REDACAO', 'EST_RENDA_PER_CAP', 'EST_CELULAR_PER_CAP', 'EST_COMP_PER_CAP']
perfil_clusters = df_cluster_clean.groupby('CLUSTER')[columns].agg(
    ['mean', 'std', 'count']
)

display(perfil_clusters.loc[:, (slice(None), 'mean')].droplevel(1, axis=1).round(2))

display(perfil_clusters.loc[:, (slice(None), 'count')].droplevel(1, axis=1).iloc[:, 0].rename('N_Observacoes'))

Plot de Gráfico Radar

In [ ]:
centroides_padronizados = pd.DataFrame(
    kmeans_final.cluster_centers_, 
    columns=columns, 
    index=[f'Cluster {i}' for i in range(K_FINAL)]
)

# 2. Configuração do Radar
categorias = columns
N = len(categorias)

# O ângulo de cada eixo no gráfico de radar
angulos = [n / float(N) * 2 * np.pi for n in range(N)]
angulos += angulos[:1]

plt.figure(figsize=(10, 10))

# 3. Plotagem para cada Cluster
for i, cluster in centroides_padronizados.iterrows():
    valores = cluster.values.flatten().tolist()
    valores += valores[:1] # Fecha o círculo

    ax = plt.subplot(111, polar=True)
    ax.plot(angulos, valores, linewidth=2, linestyle='solid', label=i)
    ax.fill(angulos, valores, alpha=0.1)

# 4. Ajustes Finais do Gráfico
ax.set_xticks(angulos[:-1])
ax.set_xticklabels([c.replace('NU_NOTA_', '') for c in categorias]) # Rótulos mais limpos
ax.set_yticks([-1.0, 0.0, 1.0])
ax.set_yticklabels(['-1 DP', 'Média (0)', '+1 DP'])
ax.set_title('Perfil de Desempenho dos Clusters (Z-Score)', size=16, y=1.1)
ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
plt.show()